In [10]:
import pandas as pd 

In [12]:
import re 
import numpy as np

In [15]:
df = pd.read_excel(r'D:\python\File\Customer Call List.xlsx')

In [17]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,Not_Useful_Column
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No,True
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes,False
2,1003,Walter,/White,7066950392,298 Drugs Driveway,N,NaN,True
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y,True
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,Y,No,True
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes,True
6,1007,Jeff,Winger,NaN,1209 South Street,No,No,False
7,1008,Sherlock,Holmes,876|678|3469,98 Clue Drive,N,No,False
8,1009,Gandalf,NaN,N/a,123 Middle Earth,Yes,NaN,False
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No,True


In [19]:
df = df.drop_duplicates()

In [21]:
df = df.drop(columns=["Not_Useful_Column"], errors="ignore") 

In [23]:
def clean_name(name):

    if pd.isna(name) or not isinstance(name, str):
        return np.nan
        
    # Loại bỏ khoảng trắng ở đầu và cuối
    name = name.strip()

     # Kiểm tra nếu tên là "Na", "N/a", "N/A", "na", "Na/n" và các chuỗi tương tự, trả về NaN
    if re.match(r'^(na|n/a|n/a|na/n|n)$', name, re.IGNORECASE):
        return np.nan
        
    # Xóa các ký tự đặc biệt và số, chỉ giữ chữ cái và dấu cách
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    
    # Loại bỏ khoảng trắng thừa giữa các từ
    name = re.sub(r'\s+', ' ', name)
    
    # Kiểm tra nếu chỉ là chuỗi rỗng sau khi làm sạch, trả về NaN
    if name == '':
        return np.nan

    # Kiểm tra nếu tên là một chuỗi số hoặc tên chỉ có một ký tự, trả về NaN
    if name.isdigit() or len(name) == 1:
        return np.nan
    
    # Chuyển chữ cái đầu mỗi từ thành chữ hoa, còn lại là chữ thường
    name = name.title()
    
    return name

In [25]:
# Áp dụng hàm clean_name cho các cột First_Name và Last_Name
df['First_Name'] = df['First_Name'].apply(clean_name)
df['Last_Name'] = df['Last_Name'].apply(clean_name)

In [28]:
def clean_phone(phone):
    # Kiểm tra nếu giá trị là NaN hoặc không phải chuỗi, trả về NaN
    if pd.isna(phone) or not isinstance(phone, str):
        return np.nan
    
    # Loại bỏ tất cả các ký tự không phải là số
    phone = re.sub(r'[^0-9]', '', phone)
    
    # Kiểm tra nếu số điện thoại có đủ 10 chữ số
    if len(phone) != 10:
        return np.nan  # Hoặc bạn có thể chọn trả về một giá trị mặc định như 'Invalid'

    # Định dạng lại số điện thoại về dạng 'XXX-XXX-XXXX'
    phone = f'{phone[:3]}-{phone[3:6]}-{phone[6:]}'
    
    return phone

In [30]:
df['Phone_Number'] = df['Phone_Number'].apply(clean_phone)

In [33]:
def clean_address(address):
    # Kiểm tra nếu giá trị là NaN hoặc không phải chuỗi, trả về NaN
    if pd.isna(address) or not isinstance(address, str):
        return np.nan
    
    # Loại bỏ khoảng trắng ở đầu và cuối
    address = address.strip()
    
    # Kiểm tra nếu giá trị là "Na", "N/a", "N/A", hoặc các chuỗi tương tự, trả về NaN
    if re.match(r'^(na|n/a|n/a|na/n|n|none|null|unknown)$', address, re.IGNORECASE):
        return np.nan
    
    # Loại bỏ các ký tự không phải chữ cái, số, dấu cách, dấu phẩy và dấu chấm
    address = re.sub(r'[^a-zA-Z0-9\s,\.]', '', address)
    
    # Loại bỏ khoảng trắng thừa giữa các từ
    address = re.sub(r'\s+', ' ', address)
    
    # Kiểm tra nếu chuỗi địa chỉ còn lại là rỗng hoặc chỉ chứa số, trả về NaN
    if address == '' or address.isdigit():
        return np.nan
    
    # Kiểm tra nếu địa chỉ có quá ít thông tin (chỉ một từ hoặc chỉ là số), trả về NaN
    if len(address.split()) <= 1:
        return np.nan
    
    return address

In [35]:
df['Address'] = df['Address'].apply(clean_address)

In [38]:
def clean_payment_status(value):
    if pd.isna(value) or isinstance(value, float):  # Kiểm tra NaN hoặc giá trị không hợp lệ
        return np.nan
    value = str(value).strip().lower()  # Chuyển tất cả về chữ thường
    if value in ['yes', 'y']:
        return 'Yes'
    elif value in ['no', 'n']:
        return 'No'
    else:
        return np.nan  # Trả về NaN nếu giá trị không hợp lệ



In [40]:
# Áp dụng hàm clean_payment_status cho cả 2 cột
df['Paying Customer'] = df['Paying Customer'].apply(clean_payment_status)
df['Do_Not_Contact'] = df['Do_Not_Contact'].apply(clean_payment_status)

In [67]:
# 1. Điền giá trị cho "Paying Customer"
df['Paying Customer'] = df.apply(
    lambda row: 'No' if pd.isna(row['Paying Customer']) and pd.isna(row['Phone_Number']) else row['Paying Customer'],
    axis=1
)

df['Paying Customer'] = df.apply(
    lambda row: 'No' if row['Do_Not_Contact'] == 'Yes' else row['Paying Customer'],
    axis=1
)

# 2. Điền giá trị cho "Do_Not_Contact"
df['Do_Not_Contact'] = df.apply(
    lambda row: 'No' if pd.isna(row['Phone_Number']) and row['Paying Customer'] == 'No' else row['Do_Not_Contact'],
    axis=1
)

In [69]:
df

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,No,Yes
2,1003,Walter,White,NaN,298 Drugs Driveway,No,No
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",No,Yes
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,Yes,No
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,No,Yes
6,1007,Jeff,Winger,NaN,1209 South Street,No,No
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,No,No
8,1009,Gandalf,NaN,NaN,123 Middle Earth,Yes,No
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No


In [ ]:
print(df.iloc[:, [5,6]])

In [77]:
df.to_excel(r'D:\python\File\out_Customer.xlsx', index = False)